In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Função para carregar e limpar os dados
def carregar_e_limpar_dados():
    # 1. Carregar os datasets locais
    # Taxa de juros (Selic)
    df_selic = pd.read_csv('Taxa de juros - Selic.csv', sep=";", parse_dates=['data'], dayfirst=True)
    df_selic['data'] = pd.to_datetime(df_selic['data'], format='%d/%m/%Y')

    # Investimentos diretos no exterior (IDE)
    df_ide = pd.read_csv('Investimentos diretos no exterior - IDE - saídas.csv', sep=";", parse_dates=['data'], dayfirst=True)
    df_ide['data'] = pd.to_datetime(df_ide['data'], format='%d/%m/%Y')

    # PIB
    df_pib = pd.read_csv('Produto interno bruto em R$ correntes.csv', sep=";", parse_dates=['data'], dayfirst=True)
    df_pib['data'] = pd.to_datetime(df_pib['data'], format='%d/%m/%Y')

    # 2. Obter a cotação do dólar histórica usando yfinance
    # Cotação do Dólar (USD/BRL) entre Janeiro de 2024 e Junho de 2024
    df_dolar = yf.download('USDBRL=X', start='2024-01-01', end='2024-06-30')
    df_dolar = df_dolar[['Close']].reset_index()  # Resetando o índice para que 'Date' vire uma coluna normal
    df_dolar.rename(columns={'Date': 'data', 'Close': 'cotacao_dolar'}, inplace=True)

    # Certificar-se de que a coluna 'data' tem o formato datetime
    df_dolar['data'] = pd.to_datetime(df_dolar['data'], format='%Y-%m-%d')

    # 3. Merge dos datasets em um único dataframe com a data como chave
    df_selic['data'] = pd.to_datetime(df_selic['data'], format='%d/%m/%Y')
    df_ide['data'] = pd.to_datetime(df_ide['data'], format='%d/%m/%Y')
    df_pib['data'] = pd.to_datetime(df_pib['data'], format='%d/%m/%Y')

    # Merge dos datasets usando a coluna 'data'
    df = pd.merge(df_selic, df_ide, on='data', how='inner')
    df = pd.merge(df, df_pib, on='data', how='inner')
    df = pd.merge(df, df_dolar, on='data', how='inner')

    # 4. Limpeza de dados (remover valores nulos ou ausentes)
    df = df.dropna()

    # 5. Ajuste dos dados para análise (normalização e criação de variáveis)
    # Para o modelo, vamos usar as colunas 'valor_x', 'valor_y', 'valor' como características.
    # Certifique-se de ajustar os nomes das colunas conforme o dataset real.
    X = df[['valor_x', 'valor_y', 'valor', 'cotacao_dolar']]  # Ajuste os nomes conforme necessário
    y = df['cotacao_dolar']  # variável alvo

    return X, y, df

# Carregar os dados e treinar o modelo
X, y, df = carregar_e_limpar_dados()

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Treinar o modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
mae = np.mean(np.abs(y_pred - y_test))
rmse = np.sqrt(np.mean((y_pred - y_test) ** 2))

# Exibir os resultados
print(f'MAE (Erro Absoluto Médio): {mae}')
print(f'RMSE (Raiz do Erro Quadrático Médio): {rmse}')

# Plotar a comparação entre a cotação real e a prevista
plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test, label='Cotação Real', color='blue')
plt.plot(y_test.index, y_pred, label='Cotação Prevista', color='red', linestyle='--')
plt.title('Comparação entre a Cotação Real e a Cotação Prevista')
plt.xlabel('Data')
plt.ylabel('Cotação do Dólar (BRL)')
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed


MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)